In [16]:
# Has a computer dealer and one human player [Simplified version]
# Player goal --> Get closer to the value of 21 before the dealer does
# Value --> Total value of the face-up cards held by the dealer
# Bust --> If value > 21 [They lose]
# J,Q,K --> All count as 10; A can be 1 or 11 (Depends on the player)
# There are links detailing the exact procedure of the game

# NOTE!! This is not a fully fledged game and there are corner cases which we will ignore [Insurance, split, double down]

# Have to create a deck class, player class, dealer class
# 1) Human player places a bet
# 2) Human will have 2 cards with face up
# 3) Dealer will have one card face up and one card face down
# 4) Player turn:: Hit -> Receive one more card OR stay -> stop receiving card
#                  If hit, we check if the sum is 21 again
# 5) Dealer turn:: Hits until they either beat the player or dealer busts
#                   Bust -> Value greater than 21

# CASE_1: Player hits and value goes above 21 --> Dealer collects the bet
# CASE_2: Player hits & stays before 21 (say 19). Dealer starts to hit and his value is 20. [21>20>19] => Dealer wins
# CASE_3: Same as 2, just that the dealer busts and 2 times the bet amount is credited to the player

In [17]:
# This cell contains all the global variables and imports
# suit and rank can be declared as tuples so that it can be used by all the other classes
# value will be a dictionary which will take in the 'key' and give out the value of the card

import random

suits = ('Hearts','Diamonds','Clubs','Spades')
ranks = ('Two','Three','Four','Five','Six','Seven','Eight','Nine','Ten','Jack','Queen','King','Ace')
value_1 = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10,
         'Jack':10, 'Queen':10, 'King':10, 'Ace':1}
# The below value will be used when the user opts for Aces being assigned a value of 1 or 11
value_11 = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10,
         'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

global ByDefault11
ByDefault11 = True

In [18]:
# This cell consists of the card class

class Card:
    
    def __init__(self,suit,rank): # Constructor of the class which requires suit and rank
        self.suit = suit
        self.rank = rank
        if ByDefault11:
            self.value = value_11[rank]
        else:
            self.value = value_1[rank]
        
    def __str__(self):
        return("This card is the "+ self.suit + " of "+ self.rank)

In [19]:
# This cell consists of the deck class
# should shuffle this around (that is why random was imported) -> One method
# should able to deal a card -> Pop one card object from the deck of the card (Another method)
# This class will hold onto a list of objects of cards

class Deck:
    
    def __init__(self): # Default constructor needn't take any inputs
        self.allcards = [] # This will consists of all the card objects taking its size to 52
        
        for suit in suits:
            for rank in ranks:
                Created_Card = Card(suit,rank) # Creates an object of Card
                self.allcards.append(Created_Card) # Add this to the list of cards
    
    def getSize(self):
        return(len(self.allcards))
    
    def shuffleDeck(self):
        random.shuffle(self.allcards) # shuffle is the function from pythons built in library. This fun changes in-place
        
    def dealCard(self):
        return(self.allcards.pop()) # pops the last card and returns it

In [20]:
# This cell consists of the player class
# Enters the amount he enters the game with
# Holds the cards in his hand
# Add/remove cards during the game
# Play --> Should be the top of card (index 0) [Pop[0]]
# Add --> We should add at the bottom of the card the player holds (append method will do)
# Add multiple --> Uses .extend() method which merges the curr list with new list
#                  [Append does not work because it creates nested list]

class Player:
    
    def __init__(self,name,amount):
        self.name = name
        self.allcards = []
        self.totalamount = amount
        self.value = 0
            
    def addCards(self,new_cards):
        # There could be multiple card objects or single
        if type(new_cards) == type([]): # new cards has a list of objects
            self.allcards.extend(new_cards)
        else:
            self.allcards.append(new_cards)
        self.value = self.value + new_cards.value
    
    def removeCards(self):
        self.allcards = []
        self.value = 0
        
    def getSize(self):
        return(len(self.allcards))
        
    def __str__(self):
        return(self.name + " has "+str(self.totalamount)+ " euors" + " and has a face value of "+ str(self.value))
    
    def bet(self,betamount):
        self.totalamount = self.totalamount - betamount
        
    def isAmountPresentInAccount(self,betamount):
        return(self.totalamount >= betamount)
    
    def getAmount(self):
        return(self.totalamount)

    def getValue(self):
        return(self.value)

    def updateWin(self, amount):
        self.totalamount = self.totalamount + amount
        
    def hasBust(self):
        return (self.value > 21)

In [21]:
# This cell consists the class of the dealer
# Dealer will have unlimited money and does not require any name

class Dealer:
    
    def __init__(self):
        self.totalamount = 1000000000 # Should be unlimited
        self.allcards = []
        self.value = 0
        
    def addCards(self, cards):
        if type(cards) == type([]):
            self.allcards.extend(cards)
        else:
            self.allcards.append(cards)
        self.value = self.value + cards.value
        
    def removeCards(self):
        self.allcards = []
        self.value = 0
    
    def updateValue(self, value):
        self.value = self.value + value
        
    def hasBust(self):
        return(self.value > 21)
    
    def getValue(self):
        return(self.value)

    def getAmount(self):
        return(self.totalamount)

    def bet(self,betamount):
        self.totalamount = self.totalamount - (2*betamount)
    
    def updateWin(self, amount):
        self.totalamount = self.totalamount + amount
        
    def __str__(self):
        return("Dealer has "+str(self.totalamount) + " euors" + " has a face value of "+ str(self.value))

In [23]:
# This cell will consist of the game logic and also some functions 

print("                              START OF THE SIMPLIFIED VERSION OF BLACKJACK!!!!! \n\n ")

HasEntryBeenCorrect = True
continueGame = True

while HasEntryBeenCorrect:
    try: 
        name = input("What is your name?: ")
        print(name)
        money = int(input("How much will you be getting to the table?: "))
        print(money)
        HasEntryBeenCorrect = False
    except ValueError:
        print("Incorrect entry!!! Start again")
        
# Create the player, dealer and deck objects
player_1 = Player(name,money)
mainDealer = Dealer()

if (input(player_1.name + " do you want Ace to be of value 1 or 11? [Enter 1 for 1 and anything else for 11] : ") == '1'):
    ByDefault11 = False
else:
    ByDefault11 = True

while continueGame:     
    deckObj = Deck()
    print("\n                               --> Deck has been shuffled <--- \n")
    deckObj.shuffleDeck()
    
    playerLost = False
    DealerLost = False

    
    betamount = int(input(player_1.name + " please place your bet : "))
    # Check if the player has sufficient funds
    
    if not player_1.isAmountPresentInAccount(betamount):
        print("Sorry!! You cannot place a bet greater than what your have carried. ")
        continueGame = False
        break
    else:
        # See that the player's and the dealers amounts are on the table
        player_1.bet(betamount)
        mainDealer.bet(betamount)
        TotalAmounBet = 3 * betamount

        # One card to the dealer (Faced below) ; Dealer faced below card is ignored --> MIGHT CHANGE ONCE WE CHECK THIS AGAIN
        facedbelowcard = deckObj.dealCard()

        # One card to the player
        player1Card = deckObj.dealCard()
        print(player1Card)
        player_1.addCards(player1Card)

        # One card to the dealer (Faced above)
        DealerCard = deckObj.dealCard()
        print(DealerCard)
        mainDealer.addCards(DealerCard)

        # 2nd card for the player faced up
        player1Card = deckObj.dealCard()
        print(player1Card)
        player_1.addCards(player1Card)

        # Check for bust of the player and if so, say that he has lost
        if player_1.hasBust():
            playerLost = True
            break
        else:
            while not playerLost and not DealerLost:
                # Ask the choice 
                hitchoice = input(player_1.name + " Press h/H if you want to hit. Anything else will deem you to stay: ")
                if hitchoice == 'h' or hitchoice == 'H':
                    player1Card = deckObj.dealCard()
                    player_1.addCards(player1Card)
                    print(player1Card)
                    if player_1.hasBust():
                        playerLost = True
                    else:
                        continue
                else:
                    # In our game here, once the player decides not to HIT, the dealer will continue to hit till he loses or wins
                    while not playerLost and not DealerLost:
                        DealerCard = deckObj.dealCard()
                        print(DealerCard)
                        mainDealer.addCards(DealerCard)

                        if mainDealer.hasBust():
                            DealerLost = True
                        elif not mainDealer.hasBust() and mainDealer.getValue() > player_1.getValue():
                            # Case where the value of dealer is greater than that of the player  
                            # But the value is lesser than 21
                            playerLost = True
                        else:
                            continue


    if playerLost:
        # Need add the amount bet by the dealer and the amount bet by the player to the total value
        # Therefore 3 times the betamount
        mainDealer.updateWin(TotalAmounBet)
    elif DealerLost:
        player_1.updateWin(TotalAmounBet)
    else:
        continueGame = False
    print(player_1)
    print(mainDealer)
    
    player_1.removeCards()
    mainDealer.removeCards()
    
    print(player_1)
    print(mainDealer)
    
    doYouWantToContinue = input(player_1.name + " Another round? Press n/N for NO or we'll for another round: ")
    
    if doYouWantToContinue == 'n' or doYouWantToContinue == 'N':
        print("Your total purse at the time of exit is "+ str(player_1.getAmount()))
        continueGame = False
    else:
        continue

                              START OF THE SIMPLIFIED VERSION OF BLACKJACK!!!!! 

 
What is your name?: Shri
Shri
How much will you be getting to the table?: 100
100
Shri do you want Ace to be of value 1 or 11? [Enter 1 for 1 and anything else for 11] : 1

                               --> Deck has been shuffled <--- 

Shri please place your bet : 10
This card is the Clubs of Ten
This card is the Clubs of Four
This card is the Hearts of Five
Shri Press h/H if you want to hit. Anything else will deem you to stay: h
This card is the Spades of Queen
Shri has 90 euors and has a face value of 25
Dealer has 1000000010 euors has a face value of 4
Shri has 90 euors and has a face value of 0
Dealer has 1000000010 euors has a face value of 0
Shri Another round? Press n/N for NO or we'll for another round: h

                               --> Deck has been shuffled <--- 

Shri please place your bet : 20
This card is the Clubs of Seven
This card is the Clubs of Ace
This card is the Diamonds of E